In [ ]:
import requests
import lxml
from pprint import pprint
from bs4 import BeautifulSoup
from selenium import webdriver


# url = "https://stats.nba.com/player/1628369/boxscores-traditional/"

# page = requests.get(url,'lxml')

# soup = BeautifulSoup(page.content)
# pprint(soup.prettify()[:1000])


### NOW SCRAPING THE INDEX OF PLAYERS

In [ ]:
driver = webdriver.Chrome()
url = 'https://stats.nba.com/players/list/'
driver.get(url)

In [ ]:
soup = BeautifulSoup(driver.page_source)
soup.find_all('li',class_='players-list__name')[:5]

In [ ]:
players = soup.find_all('li',class_='players-list__name')
players[0]

In [ ]:
## Get player index 
player_index = []
for player in players:
    tag = player.find('a')
    player_index.append([tag.text, tag['href']])

player_index


In [ ]:
PLAYERS = {}
for i in range(len(player_index)):

    player = player_index[i]
    PLAYERS[player[0]] = player[1].split('/')[-2]

PLAYERS

In [ ]:
import pandas as pd
df = pd.Series(PLAYERS)#,columns=['Player','ID'])#.to_csv("data/players/players-index.csv")
df =df.reset_index()
df.columns = ['player','stats.nba id']
df

In [ ]:
df.to_csv("data/players/players-index.csv")

## SAVED INDEX

In [ ]:

exp = re.compile(r'player/(\d+)/..([A-Za-z- ,]+)')
exp.findall(player_index[0][1])

In [ ]:
import re
for player in player_info:
    print(tyupe(player))
    exp = re.compile(r'player/(\d+)/..([A-Za-z- ,]+)')
    matches = re.findall(,player)
    

In [ ]:
players[0].find_all('a',href=True)[0]['href']

In [ ]:
row_tags = soup.find_all('tr')
row_tags[0].text

In [ ]:
row_tags[1].text

In [ ]:
headers = row_tags[0].text.replace('\n',',')
if headers[0]==',':
    headers = headers[1:]
headers

In [ ]:
headers = row_tags[0].text.split('\n')
row1 = row_tags[1].text.split('\n')
row2 = row_tags[2].text.split('\n')

print(len(headers),len(row1),len(row2))
print(headers,row1,row2,sep='\n\n')

In [ ]:
row_data = row_tags[1].text.replace('\n',',')
row_data

### Summary:
- Using the .text and splitting may not be the best strategy
- Try searching for tags like `td` inside of the table rows for better extraction

In [ ]:
head = row_tags[0]
print(len(head.find_all('th')))


column_names = [t.text for t in head.find_all('th')]
column_names

In [ ]:
row = row_tags[1]
len(row.find_all('td'))
[print(i,':',t.text) for i,t in enumerate(row.find_all('td'))]

In [ ]:
row_data = [t.text.strip('\n').strip() for t in row.find_all('td')]
len(row_data)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns',0)
## Get header/column names
head = row_tags[0]
column_names = [t.text for t in head.find_all('th')]

data = []
for row in row_tags[1:]:
    row_data = [t.text.strip('\n').strip() for t in row.find_all('td')]
    data.append(row_data)

df = pd.DataFrame(data,columns=column_names)
df.head(5)

In [ ]:
df.dtypes

In [ ]:

df.columns

In [ ]:
float_cols= [col for col in df.columns if '%' in col]
float_cols

In [ ]:
obj_cols= ['Match Up','W/L']
int_cols = [col for col in df.columns if (col not in float_cols) and (col not in obj_cols)]
int_cols

In [ ]:

df[~df.isna()]

In [ ]:

df.dropna(inplace=True)

In [ ]:
df[int_cols] = df[int_cols].astype('int')
df[float_cols] = df[float_cols].astype('float')
df.info()

## Now Use Selenium to Select the Last 2 Seasons and repeat

In [ ]:

sel_season = soup.find_all('select',attrs=dict(name='Season'))
sel_season[0].find_all('option')

In [ ]:
option = driver.find_element_by_css_selector('option')
option.click()

In [ ]:
season = driver.find_element_by_xpath("/html/body/main/div[2]/div/div/div[3]/div/div/div/div[1]/div[1]/div/div/label/select")
season.click()



#### Page URLS May be easier to generate than click on interface


In [ ]:
url_2019_20 = 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2019-20&SeasonType=Regular%20Season'
url_2018_19 = 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2018-19&SeasonType=Regular%20Season'
url_2017_18 = 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2017-18&SeasonType=Regular%20Season'

## SEPARATING ABOVE INTO F_STRING 
player_id = '1628369'
base_url = f"https://stats.nba.com/player/{player_id}/boxscores-traditional/"



seasons= {}
seasons[2019] = dict(start_YR = '2019', end_yr = '20')
seasons[2018] = dict(start_YR = '2018', end_yr = '19')
seasons[2017] = dict(start_YR = '2017', end_yr = '18')

## seasons have 4 digit start year and 2 digit end year
season_urls = []
for year in seasons:
    start_YR = seasons[year]['start_YR']
    end_yr = seasons[year]['end_yr']
    season_suffix = f"?Season={start_YR}-{end_yr}&SeasonType=Regular%20Season"
    full_url = base_url+season_suffix
    season_urls.append(full_url)

print(season_urls)



In [ ]:
# for url in season_urls:
#     driver.get(url)
# # driver.get(full_url)

In [ ]:
import nba_stats as ns

df = ns.get_season_df(season_urls[0])

In [ ]:
df['date'] = df['Match Up'].apply(lambda x: x.split('-')[0])
df['teams'] = df['Match Up'].apply(lambda x: x.split('-')[1])

In [ ]:
import pandas as pd
df['date'] = pd.to_datetime(df['date'])
df.set_index('date')

In [ ]:
import fsds_100719 as fs 
fs.reload(ns)

In [ ]:
df2 = ns.get_season_df(season_urls[1])
df2

In [ ]:
season_years=[2019,2018,2017]
seasons= {}
for year in season_years:
    seasons[year] = dict(start_YR = str(year), end_yr = str(year+1)[-2:])
seasons

In [ ]:
fs.reload(ns)
ns.make_season_urls_from_playerid()

In [ ]:
fs.reload(ns)

df_list = ns.get_all_seasons_df(player_id=1628369)

In [ ]:
df_list[0]

In [ ]:
df_combined = pd.concat(df_list,axis=0)
df_combined

In [ ]:
fs.reload(ns)
df_full = ns.get_all_seasons_df(player_id=1628369)
display(df_full.head(),df_full.tail())

In [ ]:
season_urls

In [ ]:
season_urls[0].split('/')[4]

# SAVING COPIED MARKDOWN TABLES TO CSVS

In [ ]:
# df_nba_stats_glossary = pd.read_clipboard(sep='|')

In [ ]:
# df_nba_stats_glossary.columns = [col.strip() for col in df_nba_stats_glossary.columns]

In [ ]:
# df_nba_stats_glossary = df_nba_stats_glossary[['COLUMN','MEANING']]

In [ ]:
# df_nba_stats_glossary.drop(0,inplace=True)
# df_nba_stats_glossary

In [ ]:
# df_nba_stats_glossary.to_csv('nba_stats_glossary.csv')
df_nba_stats_glossary=pd.read_csv('nba_stats_glossary.csv',index_col=0)
df_nba_stats_glossary

In [ ]:
# df_pts = pd.read_clipboard(sep='|')
# df_pts.columns = [col.strip() for col in df_pts.columns]
# df_pts.drop(0,inplace=True)
# df_pts = df_pts[['STAT','PTS']]
# df_pts

In [ ]:
# df_pts.to_csv('fantasy_points_conversion.csv')
df_pts = pd.read_csv('fantasy_points_conversion.csv',index_col=0)
df_pts

## SAVING PLAYER CSV
- Now in ns.save_player_df

In [ ]:
# import os,sys
# print(os.listdir())
# os.makedirs("data/players/" ,exist_ok=True)
# # print('\n\n',os.listdir('/data'))
df_full.head()

In [ ]:
filename=f"data/players/player_id_{df_full['player_id'][0]}.csv"
filename
df_full.to_csv(filename)

# CALCULATING FANTASY POINTS

In [ ]:
points = pd.read_csv('fantasy_points_conversion.csv',index_col=0)
points

In [ ]:
pts = dict(zip(points['STAT'],points['PTS']))
pts

In [ ]:
df_fantasy = df_full.copy()
pts_cols= []
for k,v in pts.items():
    col_name = k+'_pts'
    df_fantasy[col_name] = df_fantasy[k].apply(lambda x: x*pts[k])
    pts_cols.append(col_name)


df_fantasy['total_pts'] = df_fantasy.apply(lambda x: x[pts_cols].sum(),axis=1)
df_fantasy
    

In [ ]:
fs.reload(ns)
test_df = ns.calc_fantasy_stats(df_full)

In [ ]:
test_df

# WORKFLOW SUMMARY

1. Get player's nba_stats id#
2. `import nba_stats as ns`
3. Can directly run `df = ns.get_all_seasons_df(player_id= the_players_id)` (see note below: \$:)
4. `df = ns.calc_fantasy_stats(df)`
5. `ns.save_player_df(df)`

\$: Can instead run helper functions individually if don't want all seasons
- `season_urls=ns.make_season_urls_from_playerid`
- `df = ns.get_season_df(season_url[0])`





In [ ]:
import nba_stats as ns
player_id=1628369
df = ns.get_all_seasons_df(player_id=player_id)
df = ns.calc_fantasy_stats(df)
ns.save_player_df(df)


# TESTING EXTRACTING TODAY'S PLAYERS FROM NBA STATS

In [ ]:

player_index = pd.read_csv('data/players/players-index.csv',index_col=0)
player_index.set_index('player',inplace=True)

In [ ]:
df_game = pd.read_csv('data/AFTER-HOURS-MULTI-FanDuel-NBA-2019-12-27-42081-players-list.csv')
df_game

In [ ]:
df_game['nba_name'] = df_game['Last Name'] +', '+df_game['First Name']
df_game

In [ ]:
def lookup_player(x):
    if x in player_index.index:
        return player_index.loc[x]
    else:
        print(f"{x} not in player index")

df_game['nba_id'] = df_game['nba_name'].apply(lookup_player)
df_game

In [ ]:
todays_players= list(df_game['nba_name'])
list(sorted(todays_players))

In [ ]:
player_index.index

In [ ]:
for player in todays_players:
    print(player,player in player_index.index,sep='\t\t')